# Analysis Template: Michaelis-Menten Kinetics
Updated 5/7/24 DFM

In [ ]:
#enables autoreloding of modules
%load_ext autoreload
%autoreload 2

from htbam_db_api.htbam_db_api import LocalHtbamDBAPI
from htbam_analysis.analysis.experiment import HTBAMExperiment

#enable inline plotting of matplotlib figures
%matplotlib inline

#set the figure format to SVG
%config InlineBackend.figure_format = 'svg'

## 1. Connect DB Api

In [ ]:
### PARAMETERS:
EGFP_SLOPE = 91900.03
EGFP_SLOPE_CONC_UNITS = 'nM' #RFU/nM

root = '/Volumes/DuncanSSD/20240513/data_analysis/'
db_conn = LocalHtbamDBAPI(standard_curve_data_path= root + 'd1_NADPH_250ms_StandardSeries_Analysis.csv', standard_name="NADPH", standard_substrate="NADPH", standard_units="uM",
                         kinetic_data_path= root+ 'd1_250ms_TitrationSeries_Analysis.csv', kinetic_name="ADP", kinetic_substrate="ADP", kinetic_units="uM")

htbam_experiment = HTBAMExperiment(db_conn)

In [ ]:
db_conn.add_run_data(run_type="standard", data_path=root+"d1_NADPH_500ms_StandardSeries_Analysis.csv", run_name="NADPH_500ms", run_substrate="NADPH", run_units="uM")
db_conn.add_run_data(run_type="standard", data_path=root+"d1_NADPH_1000ms_StandardSeries_Analysis.csv", run_name="NADPH_1000ms", run_substrate="NADPH", run_units="uM")

In [ ]:
db_conn.add_run_data(run_type="kinetic", data_path=root+"d1_500ms_TitrationSeries_Analysis.csv", run_name="ADP_500ms", run_substrate="ADP", run_units="uM")
db_conn.add_run_data(run_type="kinetic", data_path=root+"d1_1000ms_TitrationSeries_Analysis.csv", run_name="ADP_1000ms", run_substrate="ADP", run_units="uM")

## 2. Standards

In [ ]:
htbam_experiment.fit_standard_curve('standard_0')
htbam_experiment.fit_standard_curve('standard_1')
htbam_experiment.fit_standard_curve('standard_2', high_conc_idx=-1)


In [ ]:
htbam_experiment.plot_standard_curve_chip('standard_2')

## 3. Fit Initial Rates

In [ ]:
htbam_experiment.fit_initial_rates('kinetic_0', 
                                   'standard_0',
                                   max_rxn_perc=20,
                                   starting_timepoint_index=0,
                                   max_rxn_time=200,
                                   #substrate_conc=60 # Only specified if you have a fixed substrate concentration
                                   )



In [ ]:
htbam_experiment.plot_initial_rates_chip('kinetic_0', 
    time_to_plot=.5, 
    )

In [ ]:
htbam_experiment.fit_initial_rates('kinetic_1', 
                                   'standard_1',
                                   max_rxn_perc=20,
                                   starting_timepoint_index=0,
                                   max_rxn_time=200,
                                   #substrate_conc=60 # Only specified if you have a fixed substrate concentration
                                   )



In [ ]:
htbam_experiment.plot_initial_rates_chip('kinetic_1', 
    time_to_plot=.5, 
    )

In [ ]:
htbam_experiment.fit_initial_rates('kinetic_2', 
                                   'standard_2',
                                   max_rxn_perc=20,
                                   starting_timepoint_index=0,
                                   max_rxn_time=200,
                                   #substrate_conc=60 # Only specified if you have a fixed substrate concentration
                                   )



In [ ]:
htbam_experiment.plot_initial_rates_chip('kinetic_2', 
    time_to_plot=.5, 
    )

## 4. Filter initial rates

In [ ]:
htbam_experiment.compute_enzyme_concentration('kinetic_0', EGFP_SLOPE)
htbam_experiment.compute_enzyme_concentration('kinetic_1', EGFP_SLOPE)
htbam_experiment.compute_enzyme_concentration('kinetic_2', EGFP_SLOPE)

In [ ]:
htbam_experiment.filter_initial_rates('kinetic_0',
                                      'standard_0',
                                      standard_curve_r2_cutoff = 0.98,
                                      expression_threshold = 1.0,
                                      initial_rate_R2_threshold = 0.5, 
                                      positive_initial_slope_filter = True)

In [ ]:
htbam_experiment.filter_initial_rates('kinetic_1',
                                      'standard_1',
                                      standard_curve_r2_cutoff = 0.98,
                                      expression_threshold = 1.0,
                                      initial_rate_R2_threshold = 0.5, 
                                      positive_initial_slope_filter = True)

In [ ]:
htbam_experiment.filter_initial_rates('kinetic_2',
                                      'standard_2',
                                      standard_curve_r2_cutoff = 0.98,
                                      expression_threshold = 1.0,
                                      initial_rate_R2_threshold = 0.5, 
                                      positive_initial_slope_filter = True)

In [ ]:
htbam_experiment.plot_filtered_initial_rates_chip('kinetics_0')

In [ ]:
htbam_experiment.combine_filtered_assays(['kinetic_0', 'kinetic_1', 'kinetic_2'])

In [ ]:
htbam_experiment.remove_run('kinetic_3')

## 5. Fit Michaelis-Menten:

In [ ]:
substrate_conc_unit = htbam_experiment.get_concentration_units('standard_0')
if  substrate_conc_unit != EGFP_SLOPE_CONC_UNITS: print('Substrate concentration units do not match EGFP standard curve units! \n{} != {}'.format(substrate_conc_unit, EGFP_SLOPE_CONC_UNITS))

unit_converstion = 0.001 #convert FROM eGFP units TO substrate units (in this case, nM to uM)


In [ ]:
htbam_experiment.fit_mm('kinetic_3', enzyme_conc_conversion=unit_converstion)

In [ ]:
htbam_experiment.filter_mm('kinetic_3',
                            z_score_threshold_mm = 1.5,
                            z_score_threshold_expression = 1.5,
                            save_intermediate_data = False)

In [ ]:
htbam_experiment.plot_filtered_mm('kinetic_3', 
                                  enzyme_conc_conversion=unit_converstion,
                                  show_average_fit=True)

## 6. Export to CSV

In [ ]:
htbam_experiment.export_mm_result_csv(path_to_save=root,
                                        run_name='kinetics_0')